In [18]:
from pyspark import sql
from pyspark import SparkContext

#from pyspark.sql.SQLContext import createDataFrame
#from pyspark.sql.SparkSession import createDataFrame
from pyspark.sql import SparkSession, SQLContext,        #class

sc = SparkContext()

In [5]:
# create departments
department1 = sql.Row(id = '123456', name='CS')
department2 = sql.Row(id = '789012', name='ME')
department3 = sql.Row(id = '234567', name='MM')
department4 = sql.Row(id = '456789', name='EE')
# Employee
Employee = sql.Row('firstName', 'lastName', 'email', 'salary')
employee1 = Employee('erxiao','wang', 'xxx@berkekey.edu', 100000)
employee2 = Employee('jh','wang', 'xxxx@standord.edu', 120000)
employee3 = Employee('tim', None, 'xxxx@cit.edu', 140000)
employee4 = Employee(None,'ng', 'xxxx@mit.edu', 160000)
# combine department and employee
departmentWithemployee1 = sql.Row(department = department1, employees =[employee1, employee2])
departmentWithemployee2 = sql.Row(department = department2, employees = [employee3, employee4])
departmentWithemployee3 = sql.Row(department = department3, employees = [employee1, employee4])
departmentWithemployee4 = sql.Row(department = department4, employees = [employee2, employee3])
# df1

In [23]:
sqlConctent = SQLContext(sc)
sparkSession = SparkSession(sc)
df1 = sqlConctent.createDataFrame([departmentWithemployee1, departmentWithemployee2])
df2 = sparkSession.createDataFrame([departmentWithemployee2, departmentWithemployee4])
df3 = sqlConctent.createDataFrame([departmentWithemployee3, departmentWithemployee4])

## print pyspark df object 

In [26]:
df1.collect()

[Row(department=Row(id='123456', name='CS'), employees=[Row(firstName='erxiao', lastName='wang', email='xxx@berkekey.edu', salary=100000), Row(firstName='jh', lastName='wang', email='xxxx@standord.edu', salary=120000)]),
 Row(department=Row(id='789012', name='ME'), employees=[Row(firstName='tim', lastName=None, email='xxxx@cit.edu', salary=140000), Row(firstName=None, lastName='ng', email='xxxx@mit.edu', salary=160000)])]

In [25]:
display(df2)

DataFrame[department: struct<id:string,name:string>, employees: array<struct<firstName:string,lastName:string,email:string,salary:bigint>>]

In [27]:
df1.toPandas()

,department,employees
0,"(123456, CS)","[(erxiao, wang, xxx@berkekey.edu, 100000), (jh..."
1,"(789012, ME)","[(tim, None, xxxx@cit.edu, 140000), (None, ng,..."


## union DF object

In [30]:
unionDF = df1.union(df3)
unionDF.collect()

[Row(department=Row(id='123456', name='CS'), employees=[Row(firstName='erxiao', lastName='wang', email='xxx@berkekey.edu', salary=100000), Row(firstName='jh', lastName='wang', email='xxxx@standord.edu', salary=120000)]),
 Row(department=Row(id='789012', name='ME'), employees=[Row(firstName='tim', lastName=None, email='xxxx@cit.edu', salary=140000), Row(firstName=None, lastName='ng', email='xxxx@mit.edu', salary=160000)]),
 Row(department=Row(id='234567', name='MM'), employees=[Row(firstName='erxiao', lastName='wang', email='xxx@berkekey.edu', salary=100000), Row(firstName=None, lastName='ng', email='xxxx@mit.edu', salary=160000)]),
 Row(department=Row(id='456789', name='EE'), employees=[Row(firstName='jh', lastName='wang', email='xxxx@standord.edu', salary=120000), Row(firstName='tim', lastName=None, email='xxxx@cit.edu', salary=140000)])]

In [31]:
unionDF.toPandas()

,department,employees
0,"(123456, CS)","[(erxiao, wang, xxx@berkekey.edu, 100000), (jh..."
1,"(789012, ME)","[(tim, None, xxxx@cit.edu, 140000), (None, ng,..."
2,"(234567, MM)","[(erxiao, wang, xxx@berkekey.edu, 100000), (No..."
3,"(456789, EE)","[(jh, wang, xxxx@standord.edu, 120000), (tim, ..."


## wirte and read pyspark DF object

In [32]:
unionDF.write.parquet('../data/uninDF.parquet')

In [34]:
parquetDF = sqlConctent.read.parquet('../data/uninDF.parquet')

In [36]:
parquetDF.toPandas()

,department,employees
0,"(123456, CS)","[(erxiao, wang, xxx@berkekey.edu, 100000), (jh..."
1,"(234567, MM)","[(erxiao, wang, xxx@berkekey.edu, 100000), (No..."
2,"(456789, EE)","[(jh, wang, xxxx@standord.edu, 120000), (tim, ..."
3,"(789012, ME)","[(tim, None, xxxx@cit.edu, 140000), (None, ng,..."


# Create spark's DataFrame object

In [46]:
eDF = sqlConctent.createDataFrame([sql.Row(a = 1, intlist=[1,2,3], mapfield={'a':'b'})])
eDFx = sqlConctent.createDataFrame([sql.Row(a = 1, mapfield={'a':'b'}, intlist=[1,2,3])])

In [47]:
eDF.collect()

[Row(a=1, intlist=[1, 2, 3], mapfield={'a': 'b'})]

In [40]:
eDFx.collect()

[Row(a=1, intlist=[1, 2, 3], mapfield={'a': 'b'})]

# explode

In [41]:
from pyspark.sql.functions import explode

In [42]:
display(eDF.select('a', explode(eDF.intlist).alias("anint")))

DataFrame[a: bigint, anint: bigint]

In [48]:
display(eDF.select(explode(eDF.mapfield).alias("k","v")))

DataFrame[k: string, v: string]

In [51]:
df = parquetDF.select('department', explode('employees').alias('e'))

In [52]:
df.collect()

[Row(department=Row(id='123456', name='CS'), e=Row(firstName='erxiao', lastName='wang', email='xxx@berkekey.edu', salary=100000)),
 Row(department=Row(id='123456', name='CS'), e=Row(firstName='jh', lastName='wang', email='xxxx@standord.edu', salary=120000)),
 Row(department=Row(id='234567', name='MM'), e=Row(firstName='erxiao', lastName='wang', email='xxx@berkekey.edu', salary=100000)),
 Row(department=Row(id='234567', name='MM'), e=Row(firstName=None, lastName='ng', email='xxxx@mit.edu', salary=160000)),
 Row(department=Row(id='456789', name='EE'), e=Row(firstName='jh', lastName='wang', email='xxxx@standord.edu', salary=120000)),
 Row(department=Row(id='456789', name='EE'), e=Row(firstName='tim', lastName=None, email='xxxx@cit.edu', salary=140000)),
 Row(department=Row(id='789012', name='ME'), e=Row(firstName='tim', lastName=None, email='xxxx@cit.edu', salary=140000)),
 Row(department=Row(id='789012', name='ME'), e=Row(firstName=None, lastName='ng', email='xxxx@mit.edu', salary=160000

# concat

In [55]:
from pyspark.sql.functions import concat


In [54]:
df = sqlConctent.createDataFrame([('abdc','123')], ['s','d'])
df.collect()

[Row(s='abdc', d='123')]

In [56]:
df.select(concat(df.s, df.d).alias('s')).collect()

[Row(s='abdc123')]

# selectxpr

In [62]:
df = parquetDF.select('department', explode('employees').alias('e'))
explodeDF = df.selectExpr("e.firstName","e.lastName","e.email","e.salary")
explodeDF.collect()

[Row(firstName='erxiao', lastName='wang', email='xxx@berkekey.edu', salary=100000),
 Row(firstName='jh', lastName='wang', email='xxxx@standord.edu', salary=120000),
 Row(firstName='erxiao', lastName='wang', email='xxx@berkekey.edu', salary=100000),
 Row(firstName=None, lastName='ng', email='xxxx@mit.edu', salary=160000),
 Row(firstName='jh', lastName='wang', email='xxxx@standord.edu', salary=120000),
 Row(firstName='tim', lastName=None, email='xxxx@cit.edu', salary=140000),
 Row(firstName='tim', lastName=None, email='xxxx@cit.edu', salary=140000),
 Row(firstName=None, lastName='ng', email='xxxx@mit.edu', salary=160000)]

#### filter

In [63]:
filterDF = explodeDF.filter(explodeDF.firstName == 'jh')
filterDF.collect()

[Row(firstName='jh', lastName='wang', email='xxxx@standord.edu', salary=120000),
 Row(firstName='jh', lastName='wang', email='xxxx@standord.edu', salary=120000)]

#### sort

In [65]:
explodeDF.sort(explodeDF.salary).collect()

[Row(firstName='erxiao', lastName='wang', email='xxx@berkekey.edu', salary=100000),
 Row(firstName='erxiao', lastName='wang', email='xxx@berkekey.edu', salary=100000),
 Row(firstName='jh', lastName='wang', email='xxxx@standord.edu', salary=120000),
 Row(firstName='jh', lastName='wang', email='xxxx@standord.edu', salary=120000),
 Row(firstName='tim', lastName=None, email='xxxx@cit.edu', salary=140000),
 Row(firstName='tim', lastName=None, email='xxxx@cit.edu', salary=140000),
 Row(firstName=None, lastName='ng', email='xxxx@mit.edu', salary=160000),
 Row(firstName=None, lastName='ng', email='xxxx@mit.edu', salary=160000)]

# col    asc

In [67]:
from pyspark.sql.functions import col, asc

In [68]:
explodeDF.filter((col('firstName')== 'jh')|(col('firstName')=='tim')).collect()

[Row(firstName='jh', lastName='wang', email='xxxx@standord.edu', salary=120000),
 Row(firstName='jh', lastName='wang', email='xxxx@standord.edu', salary=120000),
 Row(firstName='tim', lastName=None, email='xxxx@cit.edu', salary=140000),
 Row(firstName='tim', lastName=None, email='xxxx@cit.edu', salary=140000)]

#### filter

In [70]:
explodeDF.filter(col('firstName').isNull()).collect()

[Row(firstName=None, lastName='ng', email='xxxx@mit.edu', salary=160000),
 Row(firstName=None, lastName='ng', email='xxxx@mit.edu', salary=160000)]

# fillna

In [69]:
explodeDF.fillna('$$').collect()

[Row(firstName='erxiao', lastName='wang', email='xxx@berkekey.edu', salary=100000),
 Row(firstName='jh', lastName='wang', email='xxxx@standord.edu', salary=120000),
 Row(firstName='erxiao', lastName='wang', email='xxx@berkekey.edu', salary=100000),
 Row(firstName='$$', lastName='ng', email='xxxx@mit.edu', salary=160000),
 Row(firstName='jh', lastName='wang', email='xxxx@standord.edu', salary=120000),
 Row(firstName='tim', lastName='$$', email='xxxx@cit.edu', salary=140000),
 Row(firstName='tim', lastName='$$', email='xxxx@cit.edu', salary=140000),
 Row(firstName='$$', lastName='ng', email='xxxx@mit.edu', salary=160000)]

# countDistinct, count

In [71]:
from pyspark.sql.functions import countDistinct, count

In [72]:
countDistinctDF = explodeDF.select('firstName', 'lastName').groupby('firstName').agg(count('firstName'))
countDistinctDF.collect()

[Row(firstName=None, count(firstName)=0),
 Row(firstName='jh', count(firstName)=2),
 Row(firstName='tim', count(firstName)=2),
 Row(firstName='erxiao', count(firstName)=2)]

In [73]:
explodeDF.select('firstName').agg(countDistinct('firstName')).collect()

[Row(count(DISTINCT firstName)=3)]